In [18]:
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from math import pi
from qiskit.visualization import plot_histogram

In [19]:
def gcd(a, b):
    if a == 0:
        return b
     
    return gcd(b%a, a)

def c_amod15(a, power):
    """Controlled multiplication by a mod 15
    Outputs the quantum gate that corresponds to a controlled multiplication by a^power"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if a in [2,13]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [7,8]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(pi/2**(n-qubit), qubit, n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n)

def qft(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [22]:
n = 4
qc = QuantumCircuit(n*3,n*3) # multiply by 3 for the three registers

g = 13 # this is our hard-coded example, we wish to find log_13(7)

for q in range(n*2):
    qc.h(q)

reg1 = list(range(0,1*n))
reg2 = list(range(n,2*n))
reg3 = list(range(n*2,n*3))

qc.x(8) # Initialize third register to 1

# do f on third register.......
for i in range(n):
    qc.append(c_amod15(x,2**i), [i]+reg3)
    qc.append(c_amod15(g,2**i), [n+i]+reg3)
    
# Step 3: measure 3rd register
qc.measure(reg3,reg3)

# Step 4: fourier transform
qft(qc,n*2)

# Step 5: measure output again
reg12 = range(0,n*2)
qc.measure(reg12,reg12)

qc.draw(fold=-1)

aer_sim = Aer.get_backend('aer_simulator')
t_qc = transpile(qc, aer_sim)
qobj = assemble(t_qc)
results = aer_sim.run(qobj).result()
counts = results.get_counts()
plot_histogram(counts)

for output in counts:
  if counts[output] > 20: # only take prominent outputs (Quantum error)
    val = int(output, 2)
    xx = (val>>4)&0xF # first register
    yy = val&0xF # second register
    if not(xx == 0 or yy == 0):
      if(gcd(yy, 15) != 1): continue
      mm = pow(yy,-1,15) # inverse of second register
      ll = (-mm*xx)%15 # the calculation of the discrete log
      print(xx,yy,ll,pow(13, ll, 15)) # print first two registers, the log
      # calculation, and a check to see if g^ll = x

12 4 12 1
8 8 14 4
12 4 12 1
7 8 1 13
7 8 1 13
12 4 12 1
8 8 14 4
8 8 14 4
7 8 1 13
12 4 12 1
8 8 14 4
